## Accelerate Inference: Neural Network Pruning

In [7]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.layers import *

print(tf.version.VERSION)

2.10.0


In [8]:
# untar
!tar -xvzf dataset.tar.gz
# load train
train_images = pickle.load(open('train_images.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('val_images.pkl', 'rb'))
val_labels = pickle.load(open('val_labels.pkl', 'rb'))

x train_images.pkl
x train_labels.pkl
x val_images.pkl
x val_labels.pkl


In [9]:
# Define the neural network architecture (don't change this)

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [10]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 25, 25, 32)        896       
                                                                 
 activation_6 (Activation)   (None, 25, 25, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 activation_7 (Activation)   (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 11, 11, 32)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 11, 11, 32)        0         
                                                      

In [11]:
# you can use the default hyper-parameters for training, 
# and val accuracy ~59% after 25 epochs and > 63% after 50 epochs

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=32, epochs=50, 
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/50


2022-11-30 14:43:20.461072: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
/Users/kelaba/mambaforge/envs/bioimage/lib/python3.9/site-packages/keras/backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


703/703 [==============================] - 16s 22ms/step - loss: 1.5304 - accuracy: 0.2989 - val_loss: 1.3970 - val_accuracy: 0.4083
Epoch 2/50
703/703 [==============================] - 15s 21ms/step - loss: 1.3507 - accuracy: 0.4238 - val_loss: 1.2846 - val_accuracy: 0.4463
Epoch 3/50
703/703 [==============================] - 15s 21ms/step - loss: 1.2866 - accuracy: 0.4541 - val_loss: 1.2351 - val_accuracy: 0.4756
Epoch 4/50
703/703 [==============================] - 15s 21ms/step - loss: 1.2427 - accuracy: 0.4826 - val_loss: 1.1827 - val_accuracy: 0.5046
Epoch 5/50
703/703 [==============================] - 14s 20ms/step - loss: 1.2064 - accuracy: 0.5030 - val_loss: 1.1340 - val_accuracy: 0.5295
Epoch 6/50
703/703 [==============================] - 13s 19ms/step - loss: 1.1719 - accuracy: 0.5225 - val_loss: 1.1061 - val_accuracy: 0.5556
Epoch 7/50
703/703 [==============================] - 13s 19ms/step - loss: 1.1422 - accuracy: 0.5386 - val_loss: 1.1909 - val_accuracy: 0.5145
Epo

In [12]:
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 18ms/step - loss: 0.7620 - accuracy: 0.7097


In [13]:
# perform pruning here
'''
f1 pruning
stage wise pruning - set a single pruning rate for all layers in one stage
'''
def prune(model, prune_rate):
    #print("hello from pruning")
    # print(model.layers)

    indices = [] #store conv layer indices
    c = len(model.layers)
    for i in range(c):
        name = model.layers[i].name
        if(name.startswith("conv")):
            w1 = model.layers[i].get_weights()[0]
            weight = w1
            weights_dict = {}
            num_filters = len(weight[0,0,0,:])
            indices.append(i)
    
    #print(indices)

    #for i in indices:
    for i in indices: # loop thru convs
        layer = model.layers[i]
        w = layer.get_weights()[0] # shape h,w,c,f
        num_filters = w.shape[-1]
        # print(w.shape, "num filters: ", num_filters)

        weights_dict = {}
        num_delete = (int(num_filters * prune_rate)) + 1
        #print("num filters to delete: ", num_delete)

        l1_list = [] #store l1s
        for j in range(num_filters):
            l1 = np.sum(abs(w[:,]))
            filt = f"{j}"
            weights_dict[filt] = l1
            l1_list.append(l1)
        
        weights_dict_sort = sorted(weights_dict.items(), key = lambda k: k[1])
        # print(weights_dict_sort)

        delete_indices = []
        for i in range(num_delete):
            delete_indices.append((int)(weights_dict_sort[i][0]))
        
        for i in delete_indices:
            # print(layer.weights[0][:,:,:,i].shape)
            layer.weights[0][:,:,:,i].assign(0)
            # layer.weights[1][:].assign(0) #assign bias to 0?

# get the weights 
# weights = model.get_weights()

# you can use set_weights() to set some weights to zero, e.g.,
# some references for pruning techniques: https://arxiv.org/pdf/1810.05270v2.pdf, https://arxiv.org/pdf/2001.04062.pdf

# weights[7][:10]=0
# model.set_weights(weights)


In [107]:
#copy model + prune
model2 = keras.models.clone_model(model)
model2.build() # replace 10 with number of variables in input layer
model2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model2.set_weights(model.get_weights())

prune(model2, 0.5)

c = len(model2.layers)
total_weights = 0
zero_weights = 0
for i in range(c):
    name = model2.layers[i].name
    if(name.startswith("conv")):
        w1 = model2.layers[i].get_weights()[0]
        for weight in w1:
            total_weights += weight.size
            zero_weights += np.sum(weight == 0)

print(total_weights, zero_weights)

65376 39996


In [108]:
# evaluate again to see how the accuracy changes
results = model2.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 1s 18ms/step - loss: 1.6271 - accuracy: 0.2004


In [109]:
#train again with pruned

model2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model2.fit(train_images, train_labels, batch_size=32, epochs=50, 
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/50
703/703 [==============================] - 13s 18ms/step - loss: 1.3695 - accuracy: 0.4231 - val_loss: 1.2193 - val_accuracy: 0.5129
Epoch 2/50
703/703 [==============================] - 13s 18ms/step - loss: 1.2090 - accuracy: 0.5087 - val_loss: 1.1129 - val_accuracy: 0.5521
Epoch 3/50
703/703 [==============================] - 13s 18ms/step - loss: 1.1586 - accuracy: 0.5305 - val_loss: 1.1613 - val_accuracy: 0.5192
Epoch 4/50
703/703 [==============================] - 15s 21ms/step - loss: 1.1267 - accuracy: 0.5493 - val_loss: 1.0522 - val_accuracy: 0.5731
Epoch 5/50
703/703 [==============================] - 14s 20ms/step - loss: 1.1087 - accuracy: 0.5566 - val_loss: 1.0637 - val_accuracy: 0.5636
Epoch 6/50
703/703 [==============================] - 14s 20ms/step - loss: 1.0836 - accuracy: 0.5674 - val_loss: 1.0197 - val_accuracy: 0.5885
Epoch 7/50
703/703 [==============================] - 13s 18ms/step - loss: 1.0637 - accuracy: 0.5752 - val_loss: 1.0162 - val_accuracy:

In [110]:
# evaluate again to see how the accuracy changes
results = model2.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 18ms/step - loss: 0.8168 - accuracy: 0.6788


In [112]:
#get num_zero + score
total_weights = 0
zero_weights = 0
all_model2_weights = model2.get_weights()
for weight in all_model2_weights:
  total_weights += weight.size
  zero_weights += np.sum(weight == 0)

print(total_weights, zero_weights)
print(( 0.6788 + zero_weights/total_weights)/2)

592933 40114
0.37322675614276823


In [113]:
# you need to save the model's weights, naming it 'my_model_weights.h5'
model2.save_weights("my_model_weights.h5")

# # running this cell will immediately download a file called 'my_model_weights.h5'
# from google.colab import files
# files.download("my_model_weights.h5")

In [ ]:
#accuracy vs sparsity curve:
spa = [0.6]
acc = [0.6788]


In [16]:
#copy model + prune
model2 = keras.models.clone_model(model)
model2.set_weights(model.get_weights())



In [18]:
prune(model2, 0.5)

#train again with pruned

model2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model2.fit(train_images, train_labels, batch_size=32, epochs=50, 
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/50


/Users/kelaba/mambaforge/envs/bioimage/lib/python3.9/site-packages/keras/backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


703/703 [==============================] - 14s 19ms/step - loss: 1.1644 - accuracy: 0.5295 - val_loss: 0.9934 - val_accuracy: 0.6135
Epoch 2/50
703/703 [==============================] - 14s 20ms/step - loss: 1.0123 - accuracy: 0.6020 - val_loss: 0.9635 - val_accuracy: 0.6154
Epoch 3/50
703/703 [==============================] - 15s 21ms/step - loss: 0.9558 - accuracy: 0.6275 - val_loss: 0.8990 - val_accuracy: 0.6400
Epoch 4/50
703/703 [==============================] - 15s 21ms/step - loss: 0.9158 - accuracy: 0.6464 - val_loss: 0.8614 - val_accuracy: 0.6626
Epoch 5/50
703/703 [==============================] - 13s 19ms/step - loss: 0.8794 - accuracy: 0.6595 - val_loss: 0.8613 - val_accuracy: 0.6653
Epoch 6/50
703/703 [==============================] - 17s 24ms/step - loss: 0.8556 - accuracy: 0.6693 - val_loss: 0.8274 - val_accuracy: 0.6780
Epoch 7/50
703/703 [==============================] - 18s 25ms/step - loss: 0.8312 - accuracy: 0.6812 - val_loss: 0.8465 - val_accuracy: 0.6681
Epo

In [17]:
results = model2.evaluate(val_images, val_labels, batch_size=128)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.